In [1]:
import os
import sys
import scipy.io as spio
import numpy as np
import plotly.graph_objects as pgo
import re

sys.path.append("../")
from helpers.get_density import get_density
from helpers.get_spins import get_spins
from helpers.downscale import downscale_3d
from plot_multiple_3Ds import plot_multiple_3Ds
from loader import get_snap_paths, load_Psi
from helpers.grids import get_kw_square_nonzero_grid, get_radius_grid, get_space_grid
from multiprocessing import Pool

In [2]:
box_N = 96
box_L = 400.0
box_dx = box_L / box_N

In [3]:
lambdas = [-1]
runs = [1]
dtos = [1, 2, 4]
to_loads = []
for ld in lambdas:
	for run in runs:
		sr = {}
		for dto in dtos:
			sim_path = f"../out_remote/2022-09-12/4-solitons,lambda={ld},run={run},dto={dto}/"
			snap_paths = get_snap_paths(sim_path, box_L, box_N)
			for (itr, time, path) in snap_paths:
				if time != 0:
					to_loads.append((ld, run, dto, itr, time, path))


In [4]:
def load_one(x):
	(ld, run, dto, itr, time, path) = x
	try:
		Psi = load_Psi(path)
		return (ld, run, dto, itr, time, Psi)
	except:
		print(f"{path} load failed")
		return (ld, run, dto, itr, time, None)

with Pool(16) as p:
 	loadeds = p.map(load_one, to_loads)

loading ../out_remote/2022-09-12/4-solitons,lambda=-1,run=1,dto=1/snap-Psi-100-289.35.matloading ../out_remote/2022-09-12/4-solitons,lambda=-1,run=1,dto=1/snap-Psi-700-2025.46.matloading ../out_remote/2022-09-12/4-solitons,lambda=-1,run=1,dto=1/snap-Psi-400-1157.41.matloading ../out_remote/2022-09-12/4-solitons,lambda=-1,run=1,dto=1/snap-Psi-1900-5497.69.matloading ../out_remote/2022-09-12/4-solitons,lambda=-1,run=1,dto=1/snap-Psi-1300-3761.57.mat
loading ../out_remote/2022-09-12/4-solitons,lambda=-1,run=1,dto=1/snap-Psi-1000-2893.52.mat
loading ../out_remote/2022-09-12/4-solitons,lambda=-1,run=1,dto=1/snap-Psi-2500-7233.80.mat


loading ../out_remote/2022-09-12/4-solitons,lambda=-1,run=1,dto=1/snap-Psi-3400-9837.96.matloading ../out_remote/2022-09-12/4-solitons,lambda=-1,run=1,dto=1/snap-Psi-3100-8969.91.mat
loading ../out_remote/2022-09-12/4-solitons,lambda=-1,run=1,dto=1/snap-Psi-2800-8101.85.matloading ../out_remote/2022-09-12/4-solitons,lambda=-1,run=1,dto=1/snap-Psi-3700-10706.02

In [5]:
def test_for(test_ld, test_run):
	snaps = {dto: [] for dto in dtos}
	for (ld, run, dto, itr, time, Psi3) in sorted(loadeds):
		if (test_ld == ld and test_run == run):
			snaps[dto].append((time, Psi3))
	
	rhos = {dto: [(t,get_density(Psi3)) for (t,Psi3) in Psi3s] for (dto, Psi3s) in snaps.items()}
	for ((t1, rho1), (t2, rho2), (t4, rho4)) in zip(rhos[dtos[0]], rhos[dtos[1]], rhos[dtos[2]]):
		assert(t1 == t2 == t4)
		v = np.abs(rho1 - rho2) / np.abs(rho2 - rho4)
		q = np.log2(v)
		# print(np.percentile(q, 1), np.percentile(q, 10), np.percentile(q, 50), np.percentile(q, 90), np.percentile(q, 99))
		# filt = (rho1 > 1E-5) + (rho2 > 1E-5) + (rho4 > 1E-5)
		print(np.average(q))
		
test_for(-1, 1)

-1.490725157548222
-0.7499003620515404
-0.6613426081222693
-0.5910168720557446
-0.7272591239098873
-0.6287726273289581
-0.6913474092885499
-0.5242249209276075
-0.6541757044411208
-0.6791952037554473
-0.4129698107564678
-0.43163030005188374
-0.2706112534473211
0.08488040247144377
0.4457327264479509
0.7704601958213025
1.001236450886578
1.2893137607100964
1.5098662287319333
1.635315894619275
1.677763496817425
1.815708561356463
1.8415736855061302
1.949999026330694
2.0134203707982867
2.0712968706046118
2.1015166266144814
2.118742847418618
2.1291206060897463
2.1285174504891398
2.1412408859019783
2.1465318271114717
2.1515533833351865
2.158782809107394
2.164292713493855
2.1666863739503834
2.172772934659428
2.175600375163648
2.1792112174226985
2.179700422298681
2.1778568604265964
2.1731460555495037
2.169018509699886
2.1747977690855644
2.1694951981283928
2.178141232536924
2.1755242157969263
2.169994521879398
2.170169047488932
2.181857224745164
2.185485582087367
2.187588890940686
2.18689954312233